In [1]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns
    
import geopandas as gpd

import numpy as np

import rasterio

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import dask.dataframe as dd

import gc
import sys

from multiprocessing import Pool
import multiprocessing

from dask.diagnostics import ProgressBar

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Get Data
Get the temperature data for all zipcodes here.

In [2]:
def getData(weatherType,yearRange1,yearRange2):

    if weatherType == "Tmax":
        weatherVar = "temperature"
    else: 
        weatherVar = "precipitation"
    
    year = yearRange1
    filename = "../../../../../../../Volumes/backup2/dissData/prism/zipcode" + weatherType + str(year) + ".csv"
    data = dd.read_csv(filename, assume_missing=True)[['ZIP','date',weatherVar]]
    data['ZIP'] = data.ZIP.astype('int64')

    
   
    years = range(yearRange1 + 1,yearRange2 + 1)
    for year in years:
        filename = "../../../../../../../Volumes/backup2/dissData/prism/zipcode" + weatherType + str(year) + ".csv"
        tempData = dd.read_csv(filename, assume_missing=True)[['ZIP','date',weatherVar]]
        tempData['ZIP'] = tempData.ZIP.astype('int64')

        # tempData = tempData[tempData.ZIP.isin(relevantZips)]
        # tempData = tempData[~(tempData[weatherVar].isna().compute())]
        data = data.append(tempData)

    data = data[~(data[weatherVar].isna())] # .compute()
    
    return(data)

In [98]:
def getPivotQuants(weatherType, identifiers,weatherData):
    start2 = time.time()
    quants_overallByZip   = weatherData.groupby(identifiers)[weatherType].quantile(q = quantiles).reset_index().\
        rename(columns=lambda x: re.sub('level_[0-9]$','quartile',x))
    # rename(columns = {'level_1': 'quartile'})
    
    print("got grouped")
    print(time.time() - start2)
    quants_overallByZip['quartile'] = 'q_' + quants_overallByZip['quartile'].astype(str).str.slice(0,6)

    ## 

    pivot_quants_overallByZip = quants_overallByZip.pivot(index=identifiers, 
            columns='quartile', values=str(weatherType)).reset_index().\
            rename_axis(None, axis=1)
    
    pivot_quants_overallByZip['q_0.0'] = -50
    pivot_quants_overallByZip['q_1.0'] = 1000
    # print(pivot_quants_overallByZip.head())

    
    # get the quartile list
    quartileList = pivot_quants_overallByZip.iloc[:,len(identifiers):].values.tolist()
    '''quartileList[0]  = -50
    quartileList[-1] = 1000'''
    
    pivot_quants_overallByZip['quartileList']      = quartileList
    pivot_quants_overallByZip['quartileLabelList'] = [list(pivot_quants_overallByZip.columns[2:-1])] * \
                                                                    pivot_quants_overallByZip.shape[0]

    pivot_quants_overallByZip = pivot_quants_overallByZip[pivot_quants_overallByZip.\
                                                            columns.drop(list(pivot_quants_overallByZip.filter(regex='q_')))]
    print(time.time() - start2)
    # pivot_quants_overallByZip.head()

    return(pivot_quants_overallByZip)

In [94]:
quantiles = [0.0, 0.05, 0.50, 0.90, 0.95, 0.99, 1.0]

quant_labels = ['quant_0.05','quant_tossThisOne','quant_0.5_0.9',
                'quant_0.9_0.95','quant_0.95_0.99','quant_0.99_1.0']


## Run for temp and precip

In [184]:
precip = pd.read_csv("../../../../../../../Volumes/backup2/dissData/allZipsPrecip.csv").drop(columns = 'Unnamed: 0')
temp   = pd.read_csv("../../../../../../../Volumes/backup2/dissData/allZipsTemp.csv").drop(columns = 'Unnamed: 0')

temp['precipitation'] = precip.precipitation

del precip
gc.collect()

225

Get the week, month, quarter, and year for everything. The week is a little bit unusual, as it can depend on when Monday falls. But we want something a little bit different. It's referenced here: https://stackoverflow.com/questions/55889766/pandas-datetime-week-not-as-expected

In [185]:
temp['date'] = pd.to_datetime(temp['date'], format='%Y%m%d')
temp['dayOfYear'] = temp.date.dt.dayofyear
temp['week'] = temp.dayOfYear//7

In [188]:
temp.week.min()

0

In [87]:
temp['month']    = temp['date'].dt.month
temp['quarter']  = temp['date'].dt.quarter
temp['year']     = temp['date'].dt.year

In [88]:
weekly    = temp.drop(columns = {'date','month','quarter'}).groupby(['ZIP','week','year']).mean().reset_index()
monthly   = temp.drop(columns = {'date','week'}).groupby(['ZIP','month','quarter','year']).mean().reset_index()
quarterly = temp.drop(columns = {'date','month','week'}).groupby(['ZIP','quarter','year']).mean().reset_index()

In [89]:
weekly.head()

,ZIP,week,year,temperature,precipitation,dayOfYear
0,1001,0,1981,-7.465167,0.268833,3.5
1,1001,0,1982,3.875000,13.926667,3.5
2,1001,0,1983,2.708000,2.790833,3.5
3,1001,0,1984,1.857833,0.076333,3.5
4,1001,0,1985,2.270667,1.525667,3.5


In [99]:
## weekly
weeklyTemp = getPivotQuants('temperature', ['week','ZIP'], weekly).drop(columns = {'quartileLabelList'})
weeklyTemp.rename(columns = {'quartileList':     'quartileListTemperature_weeklyZip'},inplace = True)

weeklyPrecip = getPivotQuants('precipitation', ['week','ZIP'], weekly).drop(columns = {'quartileLabelList'})
weeklyPrecip.rename(columns = {'quartileList':     'quartileListPrecipitation_weeklyZip'},inplace = True)

weeklyQuartiles = weeklyTemp.merge(weeklyPrecip)

weeklyQuartiles.head()

got grouped
125.32230997085571
141.74308681488037
got grouped
117.74025893211365
131.16963601112366


,week,ZIP,quartileListTemperature_weeklyZip,quartileListPrecipitation_weeklyZip
0,0,1001,"[-50.0, -2.8099499981850387, 2.217666645844778...","[-50.0, 0.06040833244721098, 2.543583348393438..."
1,0,1002,"[-50.0, -4.439416740834711, 0.9441666593775151...","[-50.0, 0.019833333541949615, 2.42941669126351..."
2,0,1003,"[-50.0, -3.24499167005221, 1.915833269556364, ...","[-50.0, 0.02251666709780694, 2.469750006993611..."
3,0,1005,"[-50.0, -5.269758321096498, 0.4949999954551469...","[-50.0, 0.0, 2.960666716098788, 6.168416752417..."
4,0,1007,"[-50.0, -3.862775026758513, 1.4233333698163417...","[-50.0, 0.0, 2.7474999974171324, 5.13206652005..."


In [100]:
## monthly
monthlyTemp = getPivotQuants('temperature', ['month','quarter','ZIP'], monthly).drop(columns = {'quartileLabelList'})
monthlyTemp.rename(columns = {'quartileList':     'quartileListTemperature_monthlyZip'},inplace = True)

monthlyPrecip = getPivotQuants('precipitation', ['month','quarter','ZIP'], monthly).drop(columns = {'quartileLabelList'})
monthlyPrecip.rename(columns = {'quartileList':     'quartileListPrecipitation_monthlyZip'},inplace = True)

monthlyQuartiles = monthlyTemp.merge(monthlyPrecip)

monthlyQuartiles.head()

got grouped
20.7599618434906
23.68808889389038
got grouped
24.199480056762695
27.037135124206543


,month,quarter,ZIP,quartileListTemperature_monthlyZip,quartileListPrecipitation_monthlyZip
0,1,1,1001,"[-50.0, -2.6187935406161875, 1.659661311776408...","[-50.0, 0.7961016190411042, 2.577483838266363,..."
1,1,1,1002,"[-50.0, -4.254677410907441, 0.2847096796838512...","[-50.0, 0.9438774131418718, 2.5934516347103536..."
2,1,1,1003,"[-50.0, -3.2145096281363137, 1.304758073221291...","[-50.0, 0.8116258057435195, 2.4787903598720056..."
3,1,1,1005,"[-50.0, -4.841501617383574, -0.530499988055277...","[-50.0, 1.078511309527582, 2.7994032343549105,..."
4,1,1,1007,"[-50.0, -3.7949032355220087, 0.714887108413442...","[-50.0, 0.9156403169516595, 2.6491451610720915..."


In [101]:
## quarterly
quarterlyTemp = getPivotQuants('temperature', ['quarter','ZIP'], quarterly).drop(columns = {'quartileLabelList'})
quarterlyTemp.rename(columns = {'quartileList':     'quartileListTemperature_quarterlyZip'},inplace = True)

quarterlyPrecip = getPivotQuants('precipitation', ['quarter','ZIP'], quarterly).drop(columns = {'quartileLabelList'})
quarterlyPrecip.rename(columns = {'quartileList':     'quartileListPrecipitation_quarterlyZip'},inplace = True)

quarterlyQuartiles = quarterlyTemp.merge(quarterlyPrecip)

quarterlyQuartiles.head()

got grouped
5.375000953674316
6.261914014816284
got grouped
6.29679799079895
7.1825480461120605


,quarter,ZIP,quartileListTemperature_quarterlyZip,quartileListPrecipitation_quarterlyZip
0,1,1001,"[-50.0, 2.2065366838706866, 4.297861091875367,...","[-50.0, 1.496554748859434, 2.8472222221032193,..."
1,1,1002,"[-50.0, 1.070849463447101, 2.9971069086754714,...","[-50.0, 1.6742966648170514, 2.9061573954060282..."
2,1,1003,"[-50.0, 2.224399463401901, 3.8093055404334653,...","[-50.0, 1.5913055638796914, 2.800286777825148,..."
3,1,1005,"[-50.0, 0.04075832544308458, 2.067842381322239...","[-50.0, 1.8773566943841673, 2.9723576148385145..."
4,1,1007,"[-50.0, 1.5319394700229187, 3.425427769279728,...","[-50.0, 1.6826744592603706, 2.888173624678028,..."


In [28]:
quarterlyQuartiles.head()

,quarter,ZIP,quartileListTemperature_quarterlyZip,quartileListPrecipitation_quarterlyZip
0,1,1001,"[1.2713444436589876, 2.2065366838706866, 4.297...","[1.3542444386144386, 1.496554748859434, 2.8472..."
1,1,1002,"[-0.0010999896149661817, 1.070849463447101, 2....","[1.3522527351934504, 1.6742966648170514, 2.906..."
2,1,1003,"[0.8775667012151743, 2.224399463401901, 3.8093...","[1.4048571429409822, 1.5913055638796914, 2.800..."
3,1,1005,"[-0.7983888543314402, 0.04075832544308458, 2.0...","[1.4235824333442437, 1.8773566943841673, 2.972..."
4,1,1007,"[0.33490000623795596, 1.5319394700229187, 3.42...","[1.419142842860791, 1.6826744592603706, 2.8881..."


In [29]:
quantsAll = weeklyQuartiles.merge(monthlyQuartiles).merge(quarterlyQuartiles)

quantsAll.head()

,week,ZIP,quartileListTemperature_weeklyZip,quartileListPrecipitation_weeklyZip,month,quarter,quartileListTemperature_monthlyZip,quartileListPrecipitation_monthlyZip,quartileListTemperature_quarterlyZip,quartileListPrecipitation_quarterlyZip
0,0,1001,"[-7.465166668097175, -2.8099499981850387, 2.21...","[0.0, 0.06040833244721098, 2.543583348393438, ...",1,1,"[-3.1670968234539014, -2.6187935406161875, 1.6...","[0.36432257967610515, 0.7961016190411042, 2.57...","[1.2713444436589876, 2.2065366838706866, 4.297...","[1.3542444386144386, 1.496554748859434, 2.8472..."
1,0,1001,"[-7.465166668097175, -2.8099499981850387, 2.21...","[0.0, 0.06040833244721098, 2.543583348393438, ...",2,1,"[-0.49807143088297223, -0.27521787268508907, 3...","[0.16175000089320482, 1.4783482513422603, 2.23...","[1.2713444436589876, 2.2065366838706866, 4.297...","[1.3542444386144386, 1.496554748859434, 2.8472..."
2,0,1001,"[-7.465166668097175, -2.8099499981850387, 2.21...","[0.0, 0.06040833244721098, 2.543583348393438, ...",3,1,"[4.2154838724001795, 5.067606502067631, 8.2765...","[0.25170967872104344, 0.9874193588812501, 3.00...","[1.2713444436589876, 2.2065366838706866, 4.297...","[1.3542444386144386, 1.496554748859434, 2.8472..."
3,1,1001,"[-5.419714391231528, -5.053492813025202, 1.163...","[0.0, 0.17915714276688413, 2.328142845204898, ...",1,1,"[-3.1670968234539014, -2.6187935406161875, 1.6...","[0.36432257967610515, 0.7961016190411042, 2.57...","[1.2713444436589876, 2.2065366838706866, 4.297...","[1.3542444386144386, 1.496554748859434, 2.8472..."
4,1,1001,"[-5.419714391231528, -5.053492813025202, 1.163...","[0.0, 0.17915714276688413, 2.328142845204898, ...",2,1,"[-0.49807143088297223, -0.27521787268508907, 3...","[0.16175000089320482, 1.4783482513422603, 2.23...","[1.2713444436589876, 2.2065366838706866, 4.297...","[1.3542444386144386, 1.496554748859434, 2.8472..."


In [102]:
outfile =  '../../data/companyData/weeklyQuartiles.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(weeklyQuartiles, pickle_file)

In [103]:
outfile =  '../../data/companyData/monthlyQuartiles.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(monthlyQuartiles, pickle_file)

In [104]:
outfile =  '../../data/companyData/quarterlyQuartiles.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(quarterlyQuartiles, pickle_file)

In [32]:
# del temp
del weekly
del monthly
del quarterly
gc.collect()

20

# Find Quartiles by Zip

In [34]:
recentDecadeTmax   = getData("Tmax",2009,2019).compute()
recentDecadePrecip = getData("Precip",2009,2019).compute()

recentDecadeWeather = recentDecadeTmax.merge(recentDecadePrecip)
recentDecadeWeather.head()

,ZIP,date,temperature,precipitation
0,1001,20090101.0,-2.941,4.491
1,1002,20090101.0,-5.151,5.513
2,1003,20090101.0,-3.510,3.959
3,1005,20090101.0,-6.177,8.043
4,1007,20090101.0,-4.147,6.687


In [82]:
recentDecadeWeather.precipitation.max()

637.799011230469

In [35]:
del recentDecadePrecip
del recentDecadeTmax
gc.collect()

121

In [116]:
recentDecadeWeather['date'] = dd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d')
recentDecadeWeather['dayOfYear'] = recentDecadeWeather.date.dt.dayofyear
recentDecadeWeather['week'] = recentDecadeWeather.dayOfYear//7

In [119]:
recentDecadeWeather['year']       = dd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d').dt.year
recentDecadeWeather['quarter']    = dd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d').dt.quarter
recentDecadeWeather['month']      = dd.to_datetime(recentDecadeWeather['date'], format='%Y%m%d').dt.month

In [189]:
recentDecadeWeather.week.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52])

In [120]:
quantsAll.week.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52])

In [121]:
weeklyLastDec    = recentDecadeWeather.drop(columns = {'date','month','quarter'}).groupby(['ZIP','week','year']).mean().reset_index()
monthlyLastDec   = recentDecadeWeather.drop(columns = {'date','week'}).groupby(['ZIP','month','quarter','year']).mean().reset_index()
quarterlyLastDec = recentDecadeWeather.drop(columns = {'date','month','week'}).groupby(['ZIP','quarter','year']).mean().reset_index()

In [ ]:
'''recentDecadeWeather = recentDecadeWeather.astype({'ZIP': 'category',
                                                 'quarter': 'category',
                                                 'month': 'category',
                                                 'week': 'category'})'''

In [180]:
recentDecadeWeather.week.max()

52

In [53]:
ZIPs = quantsAll.ZIP.unique()
len(ZIPs)

32656

Make sure we have all the functions we need here.

In [214]:
def getZipWeeklyQuarts(zipcode):

    weatherByZipByWeek = pd.DataFrame()

    weeks = range(0,53)
    

    for week in weeks:
        tempData = weeklyLastDec[(weeklyLastDec.ZIP  == zipcode) & 
                        (weeklyLastDec.week == week)].reset_index()
        
        quantBins = weeklyQuartiles[(weeklyQuartiles.ZIP == zipcode) & 
                        (weeklyQuartiles.week == week)].reset_index()

        
        if tempData.shape[0] > 0:

            for i in range(0,len(quantBins.quartileListPrecipitation_weeklyZip[0])):
                quantBins.quartileListPrecipitation_weeklyZip[0][i] = quantBins.quartileListPrecipitation_weeklyZip[0][i] + i/10000

            tempData['temp_zipWeekQuants'] = pd.cut(tempData.temperature, 
                   bins = quantBins.quartileListTemperature_weeklyZip[0],
                   labels = quant_labels,include_lowest=True)


            tempData['precip_zipWeekQuants'] = pd.cut(tempData.precipitation, 
                   bins = quantBins.quartileListPrecipitation_weeklyZip[0],
                   labels = quant_labels,include_lowest=True)


            weatherByZipByWeek = weatherByZipByWeek.append(tempData)

    return(weatherByZipByWeek[['ZIP','year','week','precip_zipWeekQuants','temp_zipWeekQuants']])



In [215]:
def getZipMonthlyQuarts(zipcode):

    weatherByZipByMonth = pd.DataFrame()

    months = range(1,13)
    

    for month in months:
        tempData = monthlyLastDec[(monthlyLastDec.ZIP  == zipcode) & 
                        (monthlyLastDec.month == month)].reset_index()
        
        quantBins = monthlyQuartiles[(monthlyQuartiles.ZIP == zipcode) & 
                        (monthlyQuartiles.month == month)].reset_index()

        
        if tempData.shape[0] > 0:

            for i in range(0,len(quantBins.quartileListPrecipitation_monthlyZip[0])):
                quantBins.quartileListPrecipitation_monthlyZip[0][i] = quantBins.quartileListPrecipitation_monthlyZip[0][i] + i/10000

            tempData['temp_zipMonthQuants'] = pd.cut(tempData.temperature, 
                   bins = quantBins.quartileListTemperature_monthlyZip[0],
                   labels = quant_labels,include_lowest=True)


            tempData['precip_zipMonthQuants'] = pd.cut(tempData.precipitation, 
                   bins = quantBins.quartileListPrecipitation_monthlyZip[0],
                   labels = quant_labels,include_lowest=True)


            weatherByZipByMonth = weatherByZipByMonth.append(tempData)

    return(weatherByZipByMonth[['ZIP','year','month','precip_zipMonthQuants','temp_zipMonthQuants']])



In [216]:
def getZipQuarterlyQuarts(zipcode):

    weatherByZipByQuarter = pd.DataFrame()

    quarters = range(1,5)
    

    for quarter in quarters:
        tempData = quarterlyLastDec[(quarterlyLastDec.ZIP  == zipcode) & 
                        (quarterlyLastDec.quarter == quarter)].reset_index()
        
        quantBins = quarterlyQuartiles[(quarterlyQuartiles.ZIP == zipcode) & 
                        (quarterlyQuartiles.quarter == quarter)].reset_index()


        if tempData.shape[0] > 0:

            for i in range(0,len(quantBins.quartileListPrecipitation_quarterlyZip[0])):
                quantBins.quartileListPrecipitation_quarterlyZip[0][i] = quantBins.quartileListPrecipitation_quarterlyZip[0][i] + i/10000

            tempData['temp_zipQuarterQuants'] = pd.cut(tempData.temperature, 
                   bins = quantBins.quartileListTemperature_quarterlyZip[0],
                   labels = quant_labels,include_lowest=True)


            tempData['precip_zipQuarterQuants'] = pd.cut(tempData.precipitation, 
                   bins = quantBins.quartileListPrecipitation_quarterlyZip[0],
                   labels = quant_labels,include_lowest=True)


            weatherByZipByQuarter = weatherByZipByQuarter.append(tempData)

    return(weatherByZipByQuarter[['ZIP','year','quarter','precip_zipQuarterQuants','temp_zipQuarterQuants']])



Now process all the data. Try for: quarters, months, and weeks. 

Right now I think weeks will be 10x as slow as quarters, so we'll see how 

In [217]:
start = time.time()


with multiprocessing.Pool() as pool:
    zipQuarterQuarts = pool.map(getZipQuarterlyQuarts, ZIPs)

    
print(time.time() - start)


quantsByQuarterZip = pd.concat(zipQuarterQuarts)
quantsByQuarterZip.head()

215.5100440979004


,ZIP,year,quarter,precip_zipQuarterQuants,temp_zipQuarterQuants
0,1001,2009,1,quant_tossThisOne,quant_tossThisOne
1,1001,2010,1,quant_0.9_0.95,quant_0.5_0.9
2,1001,2011,1,quant_0.9_0.95,quant_tossThisOne
3,1001,2012,1,quant_tossThisOne,quant_0.99_1.0
4,1001,2013,1,quant_tossThisOne,quant_tossThisOne


In [218]:
start = time.time()


with multiprocessing.Pool() as pool:
    zipMonthQuarts = pool.map(getZipMonthlyQuarts, ZIPs)

    
print(time.time() - start)


quantsByMonthZip = pd.concat(zipMonthQuarts)
quantsByMonthZip.head()

1623.8926918506622


,ZIP,year,month,precip_zipMonthQuants,temp_zipMonthQuants
0,1001,2009,1,quant_0.5_0.9,quant_tossThisOne
1,1001,2010,1,quant_0.5_0.9,quant_0.5_0.9
2,1001,2011,1,quant_0.5_0.9,quant_tossThisOne
3,1001,2012,1,quant_0.5_0.9,quant_0.5_0.9
4,1001,2013,1,quant_tossThisOne,quant_0.5_0.9


In [219]:
start = time.time()

with multiprocessing.Pool() as pool:
    zipWeekQuarts = pool.map(getZipWeeklyQuarts, ZIPs)

print(time.time() - start)

quantsByWeekZip = pd.concat(zipWeekQuarts)
quantsByWeekZip.head()

11899.303108215332


,ZIP,year,week,precip_zipWeekQuants,temp_zipWeekQuants
0,1001,2009,0,quant_tossThisOne,quant_tossThisOne
1,1001,2010,0,quant_tossThisOne,quant_tossThisOne
2,1001,2011,0,quant_tossThisOne,quant_0.5_0.9
3,1001,2012,0,quant_tossThisOne,quant_0.5_0.9
4,1001,2013,0,quant_tossThisOne,quant_tossThisOne


In [220]:
outfile =  '../../data/companyData/quantsByQuarterZip.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(quantsByQuarterZip, pickle_file)

In [221]:
outfile =  '../../data/companyData/quantsByMonthZip.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(quantsByMonthZip, pickle_file)

In [222]:
outfile =  '../../data/companyData/quantsByWeekZip.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(quantsByWeekZip, pickle_file)

In [246]:
file = open('../../data/companyData/quantsByQuarterZip.pkl','rb')
quantsByQuarterZip = pkl.load(file)

file = open('../../data/companyData/quantsByMonthZip.pkl','rb')
quantsByMonthZip = pkl.load(file)

file = open('../../data/companyData/quantsByWeekZip.pkl','rb')
quantsByWeekZip = pkl.load(file)

Now clean up the data: 
    - rename the quartiles
    - make sure there is a year-quarter in each

In [247]:
def renameQuantiles(data, quantLabels):
    tempName   = 'precip_' + quantLabels + 'Quants'
    precipName = 'temp_'   + quantLabels + 'Quants'
    
    
    for name in [tempName,precipName]:
        print(name)
        
        data[name] = data[name].astype('str')
    
        data.loc[(data[name] == 'quant_0.05'),      name]   = '0_5'
        data.loc[(data[name] == 'quant_tossThisOne'), name] = '5_50'
        data.loc[(data[name] == 'quant_0.5_0.9'),   name]   = '50_90'
        data.loc[(data[name] == 'quant_0.9_0.95'),  name]   = '90_95'
        data.loc[(data[name] == 'quant_0.95_0.99'), name]   = '95_99'
        data.loc[(data[name] == 'quant_0.99_1.0'),  name]   = '99_1'
    
    return data



In [248]:
quantsByWeekZip     = renameQuantiles(quantsByWeekZip,    'zipWeek')
quantsByMonthZip    = renameQuantiles(quantsByMonthZip,   'zipMonth')
quantsByQuarterZip  = renameQuantiles(quantsByQuarterZip, 'zipQuarter')

precip_zipWeekQuants
temp_zipWeekQuants
precip_zipMonthQuants
temp_zipMonthQuants
precip_zipQuarterQuants
temp_zipQuarterQuants


Now get all the quarters in here.

In [249]:
quantsByWeekZip['quarter'] = (quantsByWeekZip.week // 13) + 1
quantsByWeekZip.loc[(quantsByWeekZip['quarter'] == 5), 'quarter'] = 4
# quantsByWeekZip['quarter'].value_counts()

quantsByWeekZip['yearQuarter'] = quantsByWeekZip.year.astype('str') + 'q' + quantsByWeekZip.quarter.astype('str')
quantsByWeekZip.insert(0, 'yearQuarter', quantsByWeekZip.pop('yearQuarter'))
quantsByWeekZip.drop(columns = {'year','week','quarter'}, inplace = True)

quantsByWeekZip.head()

,yearQuarter,ZIP,precip_zipWeekQuants,temp_zipWeekQuants
0,2009q1,1001,5_50,5_50
1,2010q1,1001,5_50,5_50
2,2011q1,1001,5_50,50_90
3,2012q1,1001,5_50,50_90
4,2013q1,1001,5_50,5_50


In [250]:
quantsByMonthZip['quarter'] = (quantsByMonthZip.month - 1) // 3 + 1
quantsByMonthZip.loc[(quantsByMonthZip['quarter'] == 5), 'quarter'] = 4

# quantsByMonthZip['quarter'].value_counts()

quantsByMonthZip['yearQuarter'] = quantsByMonthZip.year.astype('str') + 'q' + quantsByMonthZip.quarter.astype('str')
quantsByMonthZip.insert(0, 'yearQuarter', quantsByMonthZip.pop('yearQuarter'))
quantsByMonthZip.drop(columns = {'year','month','quarter'}, inplace = True)


quantsByMonthZip.head()

,yearQuarter,ZIP,precip_zipMonthQuants,temp_zipMonthQuants
0,2009q1,1001,50_90,5_50
1,2010q1,1001,50_90,50_90
2,2011q1,1001,50_90,5_50
3,2012q1,1001,50_90,50_90
4,2013q1,1001,5_50,50_90


In [251]:
quantsByQuarterZip['yearQuarter'] = quantsByQuarterZip.year.astype('str') + \
    'q' + quantsByQuarterZip.quarter.astype('str')

quantsByQuarterZip.insert(0, 'yearQuarter', quantsByQuarterZip.pop('yearQuarter'))
quantsByQuarterZip.drop(columns = {'year','quarter'}, inplace = True)

quantsByQuarterZip.head()

,yearQuarter,ZIP,precip_zipQuarterQuants,temp_zipQuarterQuants
0,2009q1,1001,5_50,5_50
1,2010q1,1001,90_95,50_90
2,2011q1,1001,90_95,5_50
3,2012q1,1001,5_50,99_1
4,2013q1,1001,5_50,5_50


# Counts
Look for the number of weeks, months, quarters in each quartile for the time period of interest.

In [254]:
def getCounts(quartType,df):
    df['ZIP'] = df.ZIP.astype('str')
    test = df.groupby(['ZIP','yearQuarter',quartType]).size().reset_index()
    test = test.drop_duplicates()
    test.columns = ['zip', 'yearQuarter', quartType, 'occurrences']



    testPivot = test.pivot(index=['zip','yearQuarter'], 
            columns = quartType, values= 'occurrences').reset_index().\
            rename_axis(None, axis=1)

    testPivot = testPivot.fillna(0)

    testPivot.columns.values[2:] = [quartType[:-6] + str(x) for x in testPivot.columns.values[2:]]


    return(testPivot)

Get counts for week, month, and quarter, then merge.

In [257]:
countData_weeks = getCounts(quantsByWeekZip.columns[2],quantsByWeekZip).\
    merge(getCounts(quantsByWeekZip.columns[3],quantsByWeekZip))

countData_weeks.head()


,zip,yearQuarter,precip_zipWeek0_5,precip_zipWeek50_90,precip_zipWeek5_50,precip_zipWeek90_95,precip_zipWeek95_99,precip_zipWeek99_1,temp_zipWeek0_5,temp_zipWeek50_90,temp_zipWeek5_50,temp_zipWeek90_95,temp_zipWeek95_99,temp_zipWeek99_1
0,10001,2009q1,2.0,2.0,8.0,1.0,0.0,0.0,0.0,3.0,10.0,0.0,0.0,0.0
1,10001,2009q2,0.0,5.0,5.0,2.0,0.0,1.0,2.0,2.0,7.0,2.0,0.0,0.0
2,10001,2009q3,1.0,7.0,4.0,0.0,0.0,1.0,2.0,2.0,8.0,1.0,0.0,0.0
3,10001,2009q4,1.0,7.0,6.0,0.0,0.0,0.0,1.0,6.0,7.0,0.0,0.0,0.0
4,10001,2010q1,0.0,4.0,6.0,0.0,0.0,3.0,1.0,4.0,7.0,0.0,0.0,1.0


In [258]:
countData_months = getCounts(quantsByMonthZip.columns[2],quantsByMonthZip).\
    merge(getCounts(quantsByMonthZip.columns[3],quantsByMonthZip))

countData_months.head()


,zip,yearQuarter,precip_zipMonth0_5,precip_zipMonth50_90,precip_zipMonth5_50,precip_zipMonth90_95,precip_zipMonth95_99,precip_zipMonth99_1,precip_zipMonthnan,temp_zipMonth0_5,temp_zipMonth50_90,temp_zipMonth5_50,temp_zipMonth90_95,temp_zipMonth95_99,temp_zipMonth99_1,temp_zipMonthnan
0,10001,2009q1,1.0,0.0,2.0,0.0,0.0,0.0,9.0,0.0,1.0,2.0,0.0,0.0,0.0,9.0
1,10001,2009q2,0.0,2.0,0.0,0.0,1.0,0.0,9.0,1.0,1.0,1.0,0.0,0.0,0.0,9.0
2,10001,2009q3,0.0,2.0,1.0,0.0,0.0,0.0,9.0,1.0,1.0,1.0,0.0,0.0,0.0,9.0
3,10001,2009q4,0.0,1.0,1.0,1.0,0.0,0.0,9.0,0.0,1.0,2.0,0.0,0.0,0.0,9.0
4,10001,2010q1,0.0,0.0,1.0,0.0,0.0,2.0,9.0,0.0,0.0,2.0,0.0,0.0,1.0,9.0


In [259]:
countData_quarters = getCounts(quantsByQuarterZip.columns[2],quantsByQuarterZip).\
    merge(getCounts(quantsByQuarterZip.columns[3],quantsByQuarterZip))

countData_quarters.head()


,zip,yearQuarter,precip_zipQuarter0_5,precip_zipQuarter50_90,precip_zipQuarter5_50,precip_zipQuarter90_95,precip_zipQuarter95_99,precip_zipQuarter99_1,temp_zipQuarter0_5,temp_zipQuarter50_90,temp_zipQuarter5_50,temp_zipQuarter90_95,temp_zipQuarter95_99,temp_zipQuarter99_1
0,10001,2009q1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10001,2009q2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10001,2009q3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10001,2009q4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10001,2010q1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [263]:
countData = countData_weeks.merge(countData_months).merge(countData_quarters)

countData.head()

,zip,yearQuarter,precip_zipWeek0_5,precip_zipWeek50_90,precip_zipWeek5_50,precip_zipWeek90_95,precip_zipWeek95_99,precip_zipWeek99_1,temp_zipWeek0_5,temp_zipWeek50_90,...,precip_zipQuarter5_50,precip_zipQuarter90_95,precip_zipQuarter95_99,precip_zipQuarter99_1,temp_zipQuarter0_5,temp_zipQuarter50_90,temp_zipQuarter5_50,temp_zipQuarter90_95,temp_zipQuarter95_99,temp_zipQuarter99_1
0,10001,2009q1,2.0,2.0,8.0,1.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10001,2009q2,0.0,5.0,5.0,2.0,0.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10001,2009q3,1.0,7.0,4.0,0.0,0.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10001,2009q4,1.0,7.0,6.0,0.0,0.0,0.0,1.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10001,2010q1,0.0,4.0,6.0,0.0,0.0,3.0,1.0,4.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [264]:
countData.rename(columns = {'zip': 'zipcode'}, inplace = True)

countData['year']    = countData.yearQuarter.str.slice(0,4)
countData['quarter'] = countData.yearQuarter.str.slice(5,6)

countDataRevised = countData[['zipcode','year','quarter']]

countDataRevised.head()

,zipcode,year,quarter
0,10001,2009,1
1,10001,2009,2
2,10001,2009,3
3,10001,2009,4
4,10001,2010,1


In [266]:
countData.columns

Index(['zipcode', 'yearQuarter', 'precip_zipWeek0_5', 'precip_zipWeek50_90',
       'precip_zipWeek5_50', 'precip_zipWeek90_95', 'precip_zipWeek95_99',
       'precip_zipWeek99_1', 'temp_zipWeek0_5', 'temp_zipWeek50_90',
       'temp_zipWeek5_50', 'temp_zipWeek90_95', 'temp_zipWeek95_99',
       'temp_zipWeek99_1', 'precip_zipMonth0_5', 'precip_zipMonth50_90',
       'precip_zipMonth5_50', 'precip_zipMonth90_95', 'precip_zipMonth95_99',
       'precip_zipMonth99_1', 'precip_zipMonthnan', 'temp_zipMonth0_5',
       'temp_zipMonth50_90', 'temp_zipMonth5_50', 'temp_zipMonth90_95',
       'temp_zipMonth95_99', 'temp_zipMonth99_1', 'temp_zipMonthnan',
       'precip_zipQuarter0_5', 'precip_zipQuarter50_90',
       'precip_zipQuarter5_50', 'precip_zipQuarter90_95',
       'precip_zipQuarter95_99', 'precip_zipQuarter99_1', 'temp_zipQuarter0_5',
       'temp_zipQuarter50_90', 'temp_zipQuarter5_50', 'temp_zipQuarter90_95',
       'temp_zipQuarter95_99', 'temp_zipQuarter99_1', 'year', 'quarter']

In [278]:
cdf = {}


cdf['50_90']   = ['50_90','90_95','95_99','99_1']
cdf['90_95']   = ['90_95','95_99','99_1']
cdf['95_99']   = ['95_99','99_1']
cdf['99_1']    = ['99_1']


weatherVars = ['precip_','temp_']
statVars    = ['zipWeek','zipMonth','zipQuarter']

for weatherVar in weatherVars:
    for statVar in statVars:
        print(weatherVar + statVar, "*************************")
        
        # 0.95
        for cutoff in list(cdf.keys()):
            varHere = weatherVar + statVar + cutoff
            varHereRev = varHere
            
            # print(varHere)
            
            countDataRevised[varHereRev] = 0
            
            print("****")
            print("varHereRev: ", varHereRev)
            for greaterThanCutoffs in cdf[cutoff]:
                varCDF = weatherVar + statVar + greaterThanCutoffs 
                countDataRevised[varHereRev] = countDataRevised[varHereRev] + countData[varCDF] 
                print(weatherVar + statVar + greaterThanCutoffs)


precip_zipWeek *************************
****
varHereRev:  precip_zipWeek50_90
precip_zipWeek50_90
precip_zipWeek90_95
precip_zipWeek95_99
precip_zipWeek99_1
****
varHereRev:  precip_zipWeek90_95
precip_zipWeek90_95
precip_zipWeek95_99
precip_zipWeek99_1
****
varHereRev:  precip_zipWeek95_99
precip_zipWeek95_99
precip_zipWeek99_1
****
varHereRev:  precip_zipWeek99_1
precip_zipWeek99_1
precip_zipMonth *************************
****
varHereRev:  precip_zipMonth50_90
precip_zipMonth50_90
precip_zipMonth90_95
precip_zipMonth95_99
precip_zipMonth99_1
****
varHereRev:  precip_zipMonth90_95
precip_zipMonth90_95
precip_zipMonth95_99
precip_zipMonth99_1
****
varHereRev:  precip_zipMonth95_99
precip_zipMonth95_99
precip_zipMonth99_1
****
varHereRev:  precip_zipMonth99_1
precip_zipMonth99_1
precip_zipQuarter *************************
****
varHereRev:  precip_zipQuarter50_90
precip_zipQuarter50_90
precip_zipQuarter90_95
precip_zipQuarter95_99
precip_zipQuarter99_1
****
varHereRev:  precip_zipQuart

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [279]:
countDataRevised.head()

,zipcode,year,quarter,precip_zipWeek50_90,precip_zipWeek90_95,precip_zipWeek95_99,precip_zipWeek99_1,precip_zipMonth50_90,precip_zipMonth90_95,precip_zipMonth95_99,...,temp_zipWeek95_99,temp_zipWeek99_1,temp_zipMonth50_90,temp_zipMonth90_95,temp_zipMonth95_99,temp_zipMonth99_1,temp_zipQuarter50_90,temp_zipQuarter90_95,temp_zipQuarter95_99,temp_zipQuarter99_1
0,10001,2009,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,2009,2,8.0,3.0,1.0,1.0,3.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10001,2009,3,8.0,1.0,1.0,1.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10001,2009,4,7.0,0.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10001,2010,1,7.0,3.0,3.0,3.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [280]:
countDataRevised.to_csv("../../data/companyData/weekMonthQuarter_2009to2019_allZips.csv")


In [10]:
countDataRevised.head()

,Unnamed: 0,zipcode,year,quarter,precip_zipWeek50_90,precip_zipWeek90_95,precip_zipWeek95_99,precip_zipWeek99_1,precip_zipMonth50_90,precip_zipMonth90_95,...,temp_zipWeek95_99,temp_zipWeek99_1,temp_zipMonth50_90,temp_zipMonth90_95,temp_zipMonth95_99,temp_zipMonth99_1,temp_zipQuarter50_90,temp_zipQuarter90_95,temp_zipQuarter95_99,temp_zipQuarter99_1
0,0,10001,2009,1,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,10001,2009,2,8.0,3.0,1.0,1.0,3.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10001,2009,3,8.0,1.0,1.0,1.0,2.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,10001,2009,4,7.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,10001,2010,1,7.0,3.0,3.0,3.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [22]:
countDataRevised = pd.read_csv("../../data/companyData/weekMonthQuarter_2009to2019_allZips.csv").drop(columns = {'Unnamed: 0'})

countDataRevised.astype({'zipcode': 'category',
                          'quarter': 'category',
                          'year': 'category'})

,zipcode,year,quarter,precip_zipWeek50_90,precip_zipWeek90_95,precip_zipWeek95_99,precip_zipWeek99_1,precip_zipMonth50_90,precip_zipMonth90_95,precip_zipMonth95_99,...,temp_zipWeek95_99,temp_zipWeek99_1,temp_zipMonth50_90,temp_zipMonth90_95,temp_zipMonth95_99,temp_zipMonth99_1,temp_zipQuarter50_90,temp_zipQuarter90_95,temp_zipQuarter95_99,temp_zipQuarter99_1
0,10001,2009,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,2009,2,8.0,3.0,1.0,1.0,3.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10001,2009,3,8.0,1.0,1.0,1.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10001,2009,4,7.0,0.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10001,2010,1,7.0,3.0,3.0,3.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436859,99403,2018,4,8.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1436860,99403,2019,1,7.0,3.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1436861,99403,2019,2,6.0,2.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1436862,99403,2019,3,7.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Add in the 90+ days so that we're good to go more readily.

In [23]:
quarterlyExtremes = pd.read_csv("../../data/companyData/heat_above90Quarterly.csv").drop(columns = 'Unnamed: 0')
quarterlyExtremes['year']    = quarterlyExtremes.yearQuarter.str.slice(0,4).astype('int64')
quarterlyExtremes['quarter'] = quarterlyExtremes.yearQuarter.str.slice(5,6).astype('int64')

quarterlyExtremes.rename(columns = {'extreme': 'days90Plus',
                                   'heatWave': 'streak90Plus',
                                   'ZIP': 'zipcode'}, 
                        inplace = True)

quarterlyExtremes.drop(columns = 'yearQuarter', inplace = True)

quarterlyExtremes.astype({'zipcode': 'category',
                          'quarter': 'category',
                          'year': 'category'})

,zipcode,days90Plus,streak90Plus,year,quarter
0,1001,0,0,2008,1
1,1001,4,0,2008,2
2,1001,3,0,2008,3
3,1001,0,0,2008,4
4,1001,0,0,2009,1
...,...,...,...,...,...
1567483,99403,0,0,2018,4
1567484,99403,0,0,2019,1
1567485,99403,0,0,2019,2
1567486,99403,17,0,2019,3


In [24]:
countDataRevised = countDataRevised.merge(quarterlyExtremes)

countDataRevised.head()

,zipcode,year,quarter,precip_zipWeek50_90,precip_zipWeek90_95,precip_zipWeek95_99,precip_zipWeek99_1,precip_zipMonth50_90,precip_zipMonth90_95,precip_zipMonth95_99,...,temp_zipMonth50_90,temp_zipMonth90_95,temp_zipMonth95_99,temp_zipMonth99_1,temp_zipQuarter50_90,temp_zipQuarter90_95,temp_zipQuarter95_99,temp_zipQuarter99_1,days90Plus,streak90Plus
0,10001,2009,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,10001,2009,2,8.0,3.0,1.0,1.0,3.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,10001,2009,3,8.0,1.0,1.0,1.0,2.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0
3,10001,2009,4,7.0,0.0,0.0,0.0,2.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,10001,2010,1,7.0,3.0,3.0,3.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0,0


In [25]:
countDataRevised.to_csv("../../data/companyData/latestExtremes.csv")
